In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

In [2]:
batch_size=200
learning_rate=0.01
epochs=10

train_loader=DataLoader(
    datasets.MNIST('./data/mnist_data',train=True,download=True,
                    transform=transforms.Compose([
                       transforms.RandomHorizontalFlip(),
                       transforms.RandomVerticalFlip(),
#                        transforms.RandomRotation(15), # 会改变dim???
#                        transforms.RandomRotation([90, 180, 270]),
                       transforms.Resize([32, 32]),
                       transforms.RandomCrop([28, 28]),
                       transforms.ToTensor(),
                      transforms.Normalize((0.1307,),(0.3081,))
                  ])),
    batch_size=batch_size,shuffle=True
)

test_loader=DataLoader(
    datasets.MNIST('./data/mnist_data',train=False,
                  transform=transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,),(0.3081,))
                  ])),
    batch_size=batch_size,shuffle=True
)

In [3]:
class MLP(nn.Module):
    
    def __init__(self):
        super(MLP,self).__init__()
        
        self.model=nn.Sequential(
            nn.Linear(784,200),
            nn.ReLU(inplace=True),
            nn.Linear(200,200),
            nn.ReLU(inplace=True),
            nn.Linear(200,10),
            nn.ReLU(inplace=True),
        )
        
    def forward(self,x):
        x=self.model(x)
        return x

In [4]:
device=torch.device('cuda:0')
net=MLP().to(device)
optimizer=optim.SGD(net.parameters(),lr=learning_rate)
criteon=nn.CrossEntropyLoss().to(device)

for epoch in range(epochs):
    
    for batch_idx,(data,target) in enumerate(train_loader):
        data=data.view(-1,28*28)
        data,target=data.to(device),target.to(device)
        
        logits=net(data)
        loss=criteon(logits,target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % 10 ==0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
            
    test_loss=0 
    correct=0 
    for data,target in test_loader:
        data=data.view(-1,28*28)
        data,target=data.to(device),target.to(device)
        
        logits=net(data)
        test_loss+=criteon(logits,target).item()
        
        pred=logits.argmax(dim=1)
        correct+=pred.eq(target.data).sum()
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.308267
Train Epoch: 0 [2000/60000 (3%)]	Loss: 2.310356
Train Epoch: 0 [4000/60000 (7%)]	Loss: 2.300127
Train Epoch: 0 [6000/60000 (10%)]	Loss: 2.297866
Train Epoch: 0 [8000/60000 (13%)]	Loss: 2.286626
Train Epoch: 0 [10000/60000 (17%)]	Loss: 2.291466
Train Epoch: 0 [12000/60000 (20%)]	Loss: 2.279954
Train Epoch: 0 [14000/60000 (23%)]	Loss: 2.255624
Train Epoch: 0 [16000/60000 (27%)]	Loss: 2.253483
Train Epoch: 0 [18000/60000 (30%)]	Loss: 2.254794
Train Epoch: 0 [20000/60000 (33%)]	Loss: 2.254713
Train Epoch: 0 [22000/60000 (37%)]	Loss: 2.248002
Train Epoch: 0 [24000/60000 (40%)]	Loss: 2.235116
Train Epoch: 0 [26000/60000 (43%)]	Loss: 2.214068
Train Epoch: 0 [28000/60000 (47%)]	Loss: 2.192446
Train Epoch: 0 [30000/60000 (50%)]	Loss: 2.188057
Train Epoch: 0 [32000/60000 (53%)]	Loss: 2.201826
Train Epoch: 0 [34000/60000 (57%)]	Loss: 2.142636
Train Epoch: 0 [36000/60000 (60%)]	Loss: 2.165559
Train Epoch: 0 [38000/60000 (63%)]	Loss: 2.110657
Train Epoch

Train Epoch: 5 [20000/60000 (33%)]	Loss: 1.308579
Train Epoch: 5 [22000/60000 (37%)]	Loss: 1.356074
Train Epoch: 5 [24000/60000 (40%)]	Loss: 1.424046
Train Epoch: 5 [26000/60000 (43%)]	Loss: 1.391717
Train Epoch: 5 [28000/60000 (47%)]	Loss: 1.246157
Train Epoch: 5 [30000/60000 (50%)]	Loss: 1.317883
Train Epoch: 5 [32000/60000 (53%)]	Loss: 1.440822
Train Epoch: 5 [34000/60000 (57%)]	Loss: 1.441240
Train Epoch: 5 [36000/60000 (60%)]	Loss: 1.275702
Train Epoch: 5 [38000/60000 (63%)]	Loss: 1.339650
Train Epoch: 5 [40000/60000 (67%)]	Loss: 1.364841
Train Epoch: 5 [42000/60000 (70%)]	Loss: 1.250645
Train Epoch: 5 [44000/60000 (73%)]	Loss: 1.460253
Train Epoch: 5 [46000/60000 (77%)]	Loss: 1.371591
Train Epoch: 5 [48000/60000 (80%)]	Loss: 1.268882
Train Epoch: 5 [50000/60000 (83%)]	Loss: 1.361998
Train Epoch: 5 [52000/60000 (87%)]	Loss: 1.274383
Train Epoch: 5 [54000/60000 (90%)]	Loss: 1.321511
Train Epoch: 5 [56000/60000 (93%)]	Loss: 1.367184
Train Epoch: 5 [58000/60000 (97%)]	Loss: 1.225342
